## Семинар 13: "Обучение с подкреплением 1"

ФИО: Вербин Олег

###  FrozenLake


<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>



In [18]:
!pip install gym

In [19]:
import gym
import numpy as np 

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [20]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [4]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFF
FHFH
FFFH
HFFG
observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [5]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 0
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [6]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Попробуйте походить по замерзшему озеру, не упав в дырку. 
* Каждый шаг вы с вероятностью __0.5__ будете двигаться в выбранном направлении и с вероятностью __0.5__ в случайном.
* Если упадете, используйте __env.reset()__ чтобы перезапустить __env__

In [7]:
env.reset()

0

In [8]:

env.step(action_to_i['right'])
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


## Задание: 
Следуя шаблонам функций реализуйте алгоритм Policy iteration.
Протестируйте его с помощью функции __evaluate_policy__.
Попробуйте разные значение для gamma и сравните результаты.

In [245]:
def run_episode(env, policy, gamma, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward

In [246]:
def evaluate_policy(env, policy, gamma,  n = 100):
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [247]:
# Используя фиксированную value_function и жадную стратегию получите policy

def extract_policy(v,env, gamma):
    policy=[]
    for ind in range(len(v)-1):
        best_sum=-10
        for action in env.P[ind]:
            sum_act=0
            for p,vs,_,_ in env.P[ind][action]:
                if p==1:
                    sum_act=v[vs]
                    break
                sum_act+=p*v[vs]
            sum_act*=gamma
            if sum_act>best_sum:
                best_sum=sum_act
                best_act=action
        policy.append(best_act)
    policy.append(0)
    return policy

In [248]:
# Используя фиксированную policy с помощью сэмплирования получите оценку для value_function с точностью 1e-10
def compute_policy_v(env, policy, old_v, gamma):
    eps=10
    while eps>1e-10:
        V=[]
        eps=0
        for ind,v in enumerate(old_v[:-1]):
            new_v=0
            for p,vs,_,_ in env.P[ind][policy[ind]]:
                if p==1:
                    new_v=old_v[vs]
                    break
                new_v+=p*old_v[vs]
            eps=max(abs(v-new_v*gamma),eps)
            V.append(new_v*gamma)
        V.append(1/gamma)
        old_v=V
    return V

In [250]:
def policy_iteration(env, gamma):
    max_iterations = 100000
    policy=np.zeros(16)
    old_v=np.zeros(16)
    old_v[-1]=1/gamma
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env,policy,old_v,gamma)
        new_policy = extract_policy(old_policy_v,env,gamma)
        if (np.all(policy == new_policy)):
            break

        policy = new_policy
        old_v=old_policy_v
    return policy

In [251]:
def show_policy(policy):
    ans=[]
    for p in policy:
        if p==0:
            ans.append('left')
        elif p==1:
            ans.append('down')
        elif p==2:
            ans.append('right')
        else:
            ans.append('up')
    return np.array(ans).reshape(4,4)
            

In [254]:
gamma = 1.0
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env.env, gamma=gamma)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=100)

print(policy_score)
show_policy(optimal_policy)

0.78


array([['left', 'up', 'up', 'up'],
       ['left', 'left', 'left', 'left'],
       ['up', 'down', 'left', 'left'],
       ['left', 'right', 'down', 'left']], dtype='<U5')

In [255]:
gamma = 0.95
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env.env, gamma=gamma)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=100)

print(policy_score)
show_policy(optimal_policy)

0.19842205997428855


array([['left', 'up', 'left', 'up'],
       ['left', 'left', 'left', 'left'],
       ['up', 'down', 'left', 'left'],
       ['left', 'right', 'down', 'left']], dtype='<U5')

In [256]:
gamma = 0.5
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env.env, gamma=gamma)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=100)

print(policy_score)
show_policy(optimal_policy)

0.0003745200254026355


array([['down', 'up', 'right', 'up'],
       ['left', 'left', 'left', 'left'],
       ['up', 'down', 'left', 'left'],
       ['left', 'right', 'down', 'left']], dtype='<U5')

In [257]:
gamma = 0.0001
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env.env, gamma=gamma)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=100)

print(policy_score)
show_policy(optimal_policy)

1.0000000000010003e-22


array([['down', 'up', 'right', 'up'],
       ['left', 'left', 'left', 'left'],
       ['up', 'down', 'left', 'left'],
       ['left', 'right', 'down', 'left']], dtype='<U5')

In [258]:
env.render()

  (Down)
SFFF
FHFH
FFFH
HFFG


***При изменении гаммы, стратегия меняется. Это довольно логично, потому что теперь мы максимизируем немного другую сумму. При этом значительно меняется только первая строка, это связанно с тем, что во многих клетках ему просто некуда идти (рядом дырки, как, например, во второй строке), а так-же чем ближе мы к последней клетке, тем меньше влияние гаммы на оптимальную стратегию.***